In [ ]:
%run C:\Users\Omar\Desktop\Omar_Files\Python_Analysis\Siteck\NoteBooks\2.Data_Cleaning.ipynb

In [47]:
df.head()

,date,order_id,company,company_region,product_id,product,customer_id,product_category01,product_category02,product_category03,quantity,unit,price_per_case,total_sales,year,month,month_name,day,day_name
0,2021-01-01,6141309,121,South,11829,AVO32CNV2,DMKUT,32,CNV,2,224,case,41,9184,2021,1,January,1,Friday
1,2021-01-01,6377268,131,North,11541,AVO60CNV1,QPDXK,60,CNV,1,396,case,46,18216,2021,1,January,1,Friday
2,2021-01-01,6377268,131,North,11931,AVO48CNV1,QPDXK,48,CNV,1,148,case,45,6660,2021,1,January,1,Friday
3,2021-01-01,6191867,121,South,11829,AVO32CNV2,IEHDJ,32,CNV,2,150,case,41,6150,2021,1,January,1,Friday
4,2021-01-01,6191867,121,South,11385,AVO48CNV2,IEHDJ,48,CNV,2,294,case,49,14406,2021,1,January,1,Friday


In [48]:
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error,r2_score,mean_absolute_error,accuracy_score,root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from xgboost import XGBRFRegressor
import graphviz
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV,cross_val_score,KFold
import warnings                                  
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,StandardScaler
from tqdm import tqdm

*Date preprocessing*

In [49]:
categories_encoder=LabelEncoder()
for col in df:
    if col in list(df.select_dtypes(include="object").columns):
        df[col]=categories_encoder.fit_transform(df[col])

In [50]:
df.head()

,date,order_id,company,company_region,product_id,product,customer_id,product_category01,product_category02,product_category03,quantity,unit,price_per_case,total_sales,year,month,month_name,day,day_name
0,2021-01-01,231,121,1,5,1,35,0,0,2,224,0,41,9184,0,1,4,1,0
1,2021-01-01,1437,131,0,4,5,176,2,0,1,396,0,46,18216,0,1,4,1,0
2,2021-01-01,1437,131,0,6,3,176,1,0,1,148,0,45,6660,0,1,4,1,0
3,2021-01-01,470,121,1,5,1,85,0,0,2,150,0,41,6150,0,1,4,1,0
4,2021-01-01,470,121,1,2,4,85,1,0,2,294,0,49,14406,0,1,4,1,0


*Define x and y*

In [52]:
x=df.drop(["total_sales","date","year","month","month_name","day","day_name"],axis=1)
y=df["total_sales"]

*Splitting data into training and testing subsets*

In [53]:
x_train, x_test, y_train, y_test = train_test_split(x,y ,test_size= 0.20)

print("Train data shape of X = % s and Y = % s : "%(
	x_train.shape, y_train.shape))

print("Test data shape of X = % s and Y = % s : "%(
	x_test.shape, y_test.shape))

Train data shape of X = (9070, 12) and Y = (9070,) : 
Test data shape of X = (2268, 12) and Y = (2268,) : 


*Data Scaling*

In [54]:
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [55]:
models = {
                "RandomForestRegressor": RandomForestRegressor(),
                "DecisionTreeRegressor": DecisionTreeRegressor(),
                "LinearRegression": LinearRegression(),
                "XGBRFRegressor": XGBRFRegressor(),
                "GradientBoostingRegressor":GradientBoostingRegressor(),
                "AdaBoostRegressor": AdaBoostRegressor()
                
            }

In [57]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_prediction=model.predict(x_train)
    y_test_prediction=model.predict(x_test)

    model_train_mean_absolute_percentage_error=mean_absolute_percentage_error(y_train,y_train_prediction)
    model_test_mean_absolute_percentage_error=mean_absolute_percentage_error(y_test,y_test_prediction)

    model_train_r2_score=r2_score(y_train,y_train_prediction)
    model_test_r2_score=r2_score(y_test,y_test_prediction)

    print(list(models.keys())[i])

    print("Model Performance for Train Data")
    print("mean_absolute_percentage_error : {:.4f}".format(model_train_mean_absolute_percentage_error))
    print("model_train_r2_score : {:.4f}".format(model_train_r2_score))

    print("------------------------------------------------------")

    print("Model Performance for test Data")
    print("model_test_mean_absolute_percentage_error : {:.4f}".format(model_test_mean_absolute_percentage_error))
    print("model_test_r2_score : {:.4f}".format(model_test_r2_score))


    print("="*35)
    print("\n")

RandomForestRegressor
Model Performance for Train Data
mean_absolute_percentage_error : 0.0020
model_train_r2_score : 0.9994
------------------------------------------------------
Model Performance for test Data
model_test_mean_absolute_percentage_error : 0.0052
model_test_r2_score : 0.9913


DecisionTreeRegressor
Model Performance for Train Data
mean_absolute_percentage_error : 0.0000
model_train_r2_score : 1.0000
------------------------------------------------------
Model Performance for test Data
model_test_mean_absolute_percentage_error : 0.0092
model_test_r2_score : 0.9859


LinearRegression
Model Performance for Train Data
mean_absolute_percentage_error : 0.1651
model_train_r2_score : 0.9870
------------------------------------------------------
Model Performance for test Data
model_test_mean_absolute_percentage_error : 0.1637
model_test_r2_score : 0.9837


XGBRFRegressor
Model Performance for Train Data
mean_absolute_percentage_error : 0.0885
model_train_r2_score : 0.9926
-----

In [58]:
report = {}
#report2={}
for i in tqdm(range(len(list(models)))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_prediction=model.predict(x_train)
    y_test_prediction=model.predict(x_test)

    model_train_mean_absolute_percentage_error=mean_absolute_percentage_error(y_train,y_train_prediction)
    model_test_mean_absolute_percentage_error=mean_absolute_percentage_error(y_test,y_test_prediction)

    model_train_r2_score=r2_score(y_train,y_train_prediction)
    model_test_r2_score=r2_score(y_test,y_test_prediction)

    #report[list(models.keys())[i]] = model_train_mean_absolute_percentage_error
    report[list(models.keys())[i]] = (model_train_mean_absolute_percentage_error,model_test_mean_absolute_percentage_error,model_train_r2_score,model_test_r2_score)



100%|██████████| 6/6 [00:08<00:00,  1.37s/it]


In [60]:
report_df=pd.DataFrame(report)

In [61]:
report_df

,RandomForestRegressor,DecisionTreeRegressor,LinearRegression,XGBRFRegressor,GradientBoostingRegressor,AdaBoostRegressor
0,0.002010,0.000000,0.165142,0.088539,0.025462,1.142604
1,0.005182,0.008900,0.163683,0.086991,0.025960,1.132812
2,0.999414,1.000000,0.987028,0.992622,0.999644,0.949315
3,0.991252,0.986298,0.983744,0.955072,0.990901,0.943797


In [62]:
report_df["metrices"]=["model_train_mean_absolute_percentage_error","model_test_mean_absolute_percentage_error","model_train_r2_score","model_test_r2_score"]

In [63]:
report_df

,RandomForestRegressor,DecisionTreeRegressor,LinearRegression,XGBRFRegressor,GradientBoostingRegressor,AdaBoostRegressor,metrices
0,0.002010,0.000000,0.165142,0.088539,0.025462,1.142604,model_train_mean_absolute_percentage_error
1,0.005182,0.008900,0.163683,0.086991,0.025960,1.132812,model_test_mean_absolute_percentage_error
2,0.999414,1.000000,0.987028,0.992622,0.999644,0.949315,model_train_r2_score
3,0.991252,0.986298,0.983744,0.955072,0.990901,0.943797,model_test_r2_score
